# Train an SVM Classifier on MNIST Data

In this example we will load labels and pointers to the data into a Gota dataframe.

In [1]:
import (
    "fmt"
     mnist "github.com/petar/GoMNIST"
    "github.com/kniren/gota/dataframe"
    "github.com/kniren/gota/series"
    "math/rand"
    "os"
)


In [2]:
set, err := mnist.ReadSet("../datasets/mnist/images.gz", "../datasets/mnist/labels.gz")

In [3]:
func MNISTSetToDataframe(st *mnist.Set, maxExamples int) dataframe.DataFrame {
    length := maxExamples
    if length > len(st.Images) {
        length = len(st.Images)
    }
    s := make([]string, length, length)
    l := make([]int, length, length)
    for i := 0; i < length; i++ {
        s[i] = string(st.Images[i])
        l[i] = int(st.Labels[i])
    }
    var df dataframe.DataFrame
    images := series.Strings(s)
    images.Name = "Image"
    labels := series.Ints(l)
    labels.Name = "Label"
    df = dataframe.New(images, labels)
    return df
}

In [4]:
set.Images[1]

In [5]:
df := MNISTSetToDataframe(set, 1000)

In [6]:
categories := []string{"tshirt", "trouser", "pullover", "dress", "coat", "sandal", "shirt", "shoe", "bag", "boot"}

In [7]:
func EqualsInt(s series.Series, to int) (*series.Series, error) {
    eq := make([]int, s.Len(), s.Len())
    ints, err := s.Int()
    if err != nil {
        return nil, err
    }
    for i := range ints {
        if ints[i] == to {
            eq[i] = 1
        }
    }
    ret := series.Ints(eq)
    return &ret, nil
}

In [8]:
func Split(df dataframe.DataFrame, valFraction float64) (training dataframe.DataFrame, validation dataframe.DataFrame){
    perm := rand.Perm(df.Nrow())
    cutoff := int(valFraction*float64(len(perm)))
    training = df.Subset(perm[:cutoff])
    validation = df.Subset(perm[cutoff:])
    return training, validation
}

In [9]:
training, validation := Split(df, 0.75)

In [10]:
import (
    "image"
    "bytes"
    "math"
    "github.com/gonum/stat"
    "github.com/gonum/integrate"
)

In [11]:
func NormalizeBytes(bs []byte) []float64 {
    ret := make([]float64, len(bs), len(bs))
    for i := range bs {
        ret[i] = float64(bs[i])/255.
    }
    return ret
}

In [12]:
func ImageSeriesToFloats(df dataframe.DataFrame, col string) [][]float64 {
    s := df.Col(col)
    ret := make([][]float64, s.Len(), s.Len())
    for i := 0; i < s.Len(); i++ {
        b := []byte(s.Elem(i).String())
        ret[i] = NormalizeBytes(b)
    }
    return ret
}

In [13]:
trainingImages := ImageSeriesToFloats(training, "Image")
validationImages := ImageSeriesToFloats(validation, "Image")

trainingOutputs := make([][]float64, len(trainingImages))
validationOutputs := make([][]float64, len(validationImages))

ltCol:= training.Col("Label")
for i := range trainingImages {
    l := make([]float64, len(categories), len(categories))
    val, _ := ltCol.Elem(i).Int()
    l[val] = 1
    trainingOutputs[i] = l
}

lvCol:= validation.Col("Label")
for i := range validationImages {
    l := make([]float64, len(categories), len(categories))
    val, _ := lvCol.Elem(i).Int()
    l[val] = 1
    validationOutputs[i] = l
}

## Deep Learning Classifier Using Go-Deep

In [14]:
import (
    "github.com/patrikeh/go-deep"
    "github.com/patrikeh/go-deep/training"
)

In [15]:
var (
    trainingExamples []training.Example
    validationExamples []training.Example
)
for i := range trainingImages {
    trainingExamples = append(trainingExamples, training.Example{trainingImages[i], trainingOutputs[i]})
}
for i := range validationImages {
    validationExamples = append(validationExamples, training.Example{validationImages[i], validationOutputs[i]})
}

In [16]:
network := deep.NewNeural(&deep.Config{
    // Input size: 784 in our case (number of pixels in each image)
    Inputs: len(trainingImages[0]),
    // Two hidden layers of 128 neurons each, and an output layer 10 neurons (one for each class)
    Layout: []int{128, 128, len(categories)},
    // ReLU activation to introduce some additional non-linearity
    Activation: deep.ActivationReLU,
    // We need a multi-class model
    Mode: deep.ModeMultiClass,
    // Initialise the weights of each neuron using normally distributed random numbers
    Weight: deep.NewNormal(0.5, 0.1),
    Bias: true,
})

In [17]:
// Parameters: learning rate, momentum, alpha decay, nesterov
optimizer := training.NewSGD(0.006, 0.1, 1e-6, true)
trainer := training.NewTrainer(optimizer, 1)

trainer.Train(network, trainingExamples, validationExamples, 500) // training, validation, iterations

Epochs          Elapsed         Loss (CE)       Accuracy        
---             ---             ---             ---             
1               3.3231414s      2.3441          0.08            
2               6.455526s       2.3227          0.09            
3               9.505251s       2.3121          0.09            
4               12.5443012s     2.3092          0.09            
5               15.5331261s     2.3080          0.13            
6               18.4141975s     2.3069          0.12            
7               21.2810838s     2.3036          0.13            
8               24.199852s      2.2804          0.10            
9               27.1254202s     2.0891          0.13            
10              30.0979624s     2.3016          0.08            
11              33.1896291s     2.2632          0.11            
12              36.2219657s     2.0688          0.13            
13              39.3095451s     1.9798          0.18            
14              42.842466

126             6m29.9929432s   1.0760          0.59            
127             6m32.9417875s   1.0562          0.58            
128             6m35.8308759s   1.1893          0.56            
129             6m38.6856768s   1.0786          0.58            
130             6m42.1524264s   1.1264          0.58            
131             6m45.0942436s   1.2890          0.51            
132             6m47.9009305s   1.1202          0.56            
133             6m50.8103811s   1.1070          0.57            
134             6m53.810866s    1.1360          0.58            
135             6m56.9139356s   1.1981          0.56            
136             7m0.1455446s    1.1376          0.54            
137             7m3.3730309s    1.2238          0.48            
138             7m6.7513062s    1.0789          0.55            
139             7m10.2420934s   1.1157          0.55            
140             7m13.368513s    1.3375          0.47            
141             7m16.5322

252             12m45.3407915s   0.9424          0.73            
253             12m48.2382774s   1.6405          0.69            
254             12m51.1121736s   1.0240          0.76            
255             12m53.9369158s   1.1365          0.69            
256             12m56.7718417s   1.0620          0.74            
257             12m59.5903136s   1.0010          0.75            
258             13m2.4879973s   1.0382          0.74            
259             13m5.3697603s   1.0378          0.74            
260             13m8.2142311s   1.2237          0.71            
261             13m11.022254s   1.3229          0.74            
262             13m14.4863196s   1.0079          0.74            
263             13m18.4281566s   1.0717          0.74            
264             13m21.6862611s   1.8066          0.68            
265             13m24.6630732s   1.1750          0.74            
266             13m27.4714272s   1.0809          0.74            
267           

377             19m21.6039045s   1.2853          0.76            
378             19m24.5295295s   1.3151          0.77            
379             19m27.4621895s   1.3650          0.78            
380             19m30.385587s   1.3235          0.73            
381             19m33.2471732s   1.2655          0.74            
382             19m36.4575997s   1.2157          0.78            
383             19m39.3079424s   1.3167          0.78            
384             19m42.1691277s   1.2094          0.78            
385             19m44.9936471s   1.2136          0.79            
386             19m47.8669653s   1.5024          0.78            
387             19m50.7131839s   1.3112          0.79            
388             19m53.5302093s   1.5553          0.75            
389             19m56.4045206s   1.2660          0.76            
390             19m59.3563266s   1.1618          0.79            
391             20m2.5971576s   1.2961          0.79            
392         

Calculate accuracy on validation dataset

In [18]:
func MaxIndex(f []float64) (i int){
    var (
        curr float64
        ix int = -1
    )
    for i := range f {
        if f[i] > curr {
            curr = f[i]
            ix = i
        }
    }
    return ix
}

In [19]:
validCorrect := 0.
for i := range validationImages {
    prediction := network.Predict(validationImages[i])

    if MaxIndex(prediction) == MaxIndex(validationOutputs[i]) {
        validCorrect++
    }
}
fmt.Printf("Validation Accuracy: %5.2f\n",  validCorrect/float64(len(validationImages)))

Validation Accuracy:  0.80


27 <nil>